## Introduction

This guide demonstrates the setup and interactions of a `node`, `registry`, and `provider` service using the `kodo` framework. These services are used to manage distributed flows and registries in a modular and scalable architecture. A `node` provides agentic flows and connects to a `registry`, which brokers the nodes and flows. Additionally, a `hybrid` service acts as both a `node` and a `provider`, enabling more complex workflows by integrating its flows while maintaining independent operations. 

The examples below detail how to configure and start each component, including their connections and expected behaviors, illustrated with logging output.

In [ ]:
import multiprocessing as mp
from kodo.service.node import run_service
import httpx

## Start `node` with 5 Flows

available options to `run_service`:
* `loader: Optional[str] = "kodo.worker.loader:default_loader"`
* `url:  Optional[str] = "http://localhost:3366"`
* `organization: Optional[str] = None`
* `connect: Optional[List[str]] = None`
* `registry: Optional[bool] = True`
* `feed: Optional[bool] = True`
* `cache_data: Optional[str] = "./data/cache.json"`
* `cache_reset: Optional[bool] = False`
* `screen_level: Optional[str] = "INFO"`
* `log_file: Optional[str] = "./data/kodo.log"`
* `log_file_level: Optional[str] = "DEBUG"`
* `exec_data: Optional[str] = "./data/exec"`
* `timeout: Optional[int] = 30`
* `retry: Optional[int] = 9`


In [ ]:
node = mp.Process(
    target=run_service, 
    kwargs={
        "loader": "/Users/rostf/repos/kodo-core/tests/assets/node.yaml",
        # "cache_reset": True,
        # "retry": -1  # try forever
    })
node.start()

## Start `registry`

After registry startup the node above will connect. This can take upt o 8 

In [ ]:
registry = mp.Process(
    target=run_service, 
    kwargs={
        "loader": "/Users/rostf/repos/kodo-core/tests/assets/registry.yaml",
        "cache_reset": True,
    })
registry.start()

## Start `hybrid` (node _and_ provider service)

In [ ]:
hybrid = mp.Process(
    target=run_service, 
    kwargs={
        "loader": "/Users/rostf/repos/kodo-core/tests/assets/agent50.yaml",
        "cache_reset": True
    })
hybrid.start()

## SUMMARY

A node offers flows and connects to a registry. A registry brokers nodes/flows. A provider is a registry which provides nodes/flows to other registries but does not integrate the nodes/flows from peer registries (`feed is False`).
```mermaid
    flowchart LR
        flow5[5 Flows] --> node
        node --> registry
        flow50[50 Flows] --> provider
        provider --> registry
```

In [ ]:
node_url = "http://localhost:3366"
registry_url = "http://localhost:3367"
hybrid_url = "http://localhost:3368"

assert httpx.get(f"{node_url}/").json()["registry"] is False
assert httpx.get(f"{registry_url}/").json()["registry"] is True and httpx.get(f"{registry_url}/").json()["feed"] is True
assert httpx.get(f"{hybrid_url}/").json()["registry"] is True and httpx.get(f"{hybrid_url}/").json()["feed"] is False

Retrieving the flows from `node`, `registry` and `provider` yields the following result.

In [ ]:
assert httpx.get(f"{node_url}/flows").json()["total"] == 5
assert httpx.get(f"{registry_url}/flows").json()["total"] == 55
assert httpx.get(f"{hybrid_url}/flows").json()["total"] == 50

## Details

The `registry` caches the result. On restart the `registry` delivers the nodes and flows even if the node died.

In [ ]:
node.terminate()
registry.terminate()

### registry restart

In [ ]:
registry = mp.Process(
    target=run_service, 
    kwargs={
        "loader": "/Users/rostf/repos/kodo-core/tests/assets/registry.yaml",
        "cache_reset": False,  # Default Value is False
        "retry": -1  # retry forever
    })
registry.start()

In [ ]:
assert httpx.get(f"{registry_url}/flows").json()["total"] == 55

If the node restarts itself but changes the settings or set of flows, the registry's `/reconnect` synchronizes the registry.

In [ ]:
node = mp.Process(
    target=run_service, 
    kwargs={
        "url": "http://localhost:3366",
        "registry": False,
        "feed": False,
        "organization": "Node with New Name",
        "cache_data": "./data/3366.json",
        "connect": "http://localhost:3367",
        "cache_reset": False,
        "screen_level": "DEBUG",
        "retry": -1,  # try forever
    })
node.start()

In [ ]:
assert httpx.get(f"{node_url}/").json()["organization"] == "Node with New Name"

In [ ]:
assert httpx.get(f"{node_url}/flows").json()["total"] == 0
assert httpx.get(f"{registry_url}/flows").json()["total"] == 50
assert httpx.get(f"{hybrid_url}/flows").json()["total"] == 50

## Disconnect

A node, registry or registry provider can explicitely _disconnect_ from a registry. This will remove the node and flows from the registry. The node itself still serves the flows though.

In [ ]:
resp = httpx.delete(f"{hybrid_url}/connect")
assert resp.status_code == 204

In [ ]:
assert httpx.get(f"{node_url}/flows").json()["total"] == 0
assert httpx.get(f"{registry_url}/flows").json()["total"] == 0
assert httpx.get(f"{hybrid_url}/flows").json()["total"] == 50

In [ ]:
node.terminate()

In [ ]:
registry.terminate()

In [ ]:
hybrid.terminate()